In [1]:
# Projection implementation of stochastic growth model adapted from Fabrice Collard's Matlab code, http://fabcol.free.fr/
# tested in Julia 1.5.0
# this code is part of chapter 5, "Advanced Numerical Techniques" from the book: "Introduction to Quantitative Macroeconomics using Julia"
# Academic Press - Elsevier
# for comments, email at: petre(dot)caraiani(at)gmail(dot)com

using NLsolve;

using LinearAlgebra;

# The Optimal Growth Model
# Galerkin method (AR(1) case)                 

global nbk, kmin, ksup, XK, kt;
global nba, amin, asup, XA, at;
global nstate, nodea, nodek, wmat, wgrid;

# Parameters                        
nbk   = 4;    # Degree of polynomials (capital)
nba   = 2;    # Degree of polynomials (technology shock)
ncoef = (nbk+1); # of coefficients
nodek = 20;   # of Nodes
nodea = 10;# of Nodes
nstate= 12;
                                                           
# Structural Parameters                    
delta  = 0.1;
beta   = 0.95;
alpha  = 0.3;
sigma  = 1.5;
ysk    =(1-beta*(1-delta))/(alpha*beta);
ksy    = 1/ysk;
ys     = ksy^(alpha/(1-alpha));
ks     = ys^(1/alpha);
is     = delta*ks;
cs     = ys-is;
ab     = 0;


In [2]:
function hernodes(nstate)

TOL    = sqrt(2.2204e-16);
MAXIT  = 30;
PIM4   = pi^(-1/4);

n  = nstate;
m  = (n+1)/2;
m  =  convert(Int64, floor(m));
x  = zeros(n,1);
w  = zeros(n,1);

z  = 0;
z1 = 0;
pp = 0;
for i= 1:m

# Initialize the first four roots
if i == 1; 
   z= sqrt(2*n+1) - 1.85575 * (2*n+1)^(-1/6);
elseif i== 2;
   z= z - 1.14 * (n^.426)/z;
elseif i== 3;
   z= 1.86 * z - .86 * x[1];
elseif i== 4;
   z= 1.91 * z - .91 * x[2];
else;
   z= 2 * z - x[i-2];
end;

for its= 1:MAXIT
    p1 = PIM4;
    p2 = 0;
 for j  = 1:n
    p3 = p2;
    p2 = p1;
    p1 = z*sqrt(2/j)*p2 - sqrt((j-1)/j)*p3;
 end;

    pp = p2 * sqrt(2*n);
    z1 = z;
    z  = z1 - p1/pp;
if abs(z-z1) < TOL; break; end;
end;

x[i]    = z;
x[n+1-i]= -z;
w[i]= 2/(pp*pp);
w[n+1-i]= w[i];
end;       
        
return x,w
    end;

In [3]:
# Technology shock process
rho    = 0.8;
se     = 0.2;
ma     = 0;
wgrid,wmat=hernodes(nstate);
wgrid=wgrid*sqrt(2)*se;

In [4]:
#define functions rcheb transfo itransfo
function rcheb(nn);
mod=nn-floor(nn/2)*2;
n1=floor(nn/2);
if n1==1
k=(collect(1:n1))
    elseif n1>1
k=transpose(collect(1:n1));
    end;
r1=cos.((2*k.-1)*pi/(2*nn));
r1=[r1 -r1];
if mod==1;
  r1=[r1 0];
end;
r1=vec(r1);
rr=real(sort!(r1));
return rr
end;

function transfo(x,xmin,xmax);
  z=(2*(x.-xmin)./(xmax-xmin)).-1;
return z
end;

function itransfo(x,xmin,xmax);
z=0.5*(x.+1).*(xmax.-xmin).+xmin;
return z
end;

function cheb(xx,nn);
cc=real(cos.(kron(acos.(complex(xx)),nn)))      
return cc
end;

In [5]:
# grid for the income
amin=(ma+wgrid[nstate]);
asup=(ma+wgrid[1]);
ra=rcheb(nodea);           #roots
at=itransfo(ra,amin,asup); #grid
vnba = transpose(collect(0:nba));
XA=cheb(ra,vnba)

10×3 Array{Float64,2}:
 1.0  -0.987688   0.951057
 1.0  -0.891007   0.587785
 1.0  -0.707107  -1.83697e-16
 1.0  -0.45399   -0.587785
 1.0  -0.156434  -0.951057
 1.0   0.156434  -0.951057
 1.0   0.45399   -0.587785
 1.0   0.707107   2.83277e-16
 1.0   0.891007   0.587785
 1.0   0.987688   0.951057

In [6]:
# grid for the capital stock
kmin  = log(1.2);
ksup  = log(6);
rk    = rcheb(nodek);               #roots
kt    = exp.(itransfo(rk,kmin,ksup)) #grid
vrbk = transpose(collect(0:nbk));
XK    = cheb(rk,vrbk);

In [7]:
#initial conditions
a0=[ 
  -0.23759592487257;
   0.60814488103911;
   0.03677400318790;
   0.69025680170443;
  -0.21654209984197;
   0.00551243342828;
   0.03499834613714;
  -0.00341171507904;
  -0.00449139656933;
   0.00285737302122;
  -0.00002348542016;
  -0.00011606672164;
];
param=[alpha beta delta sigma ma rho]';

In [8]:
# function makepoly
function makepoly(XA,XW);
nba   = size(XA,2);
nbw   = size(XW,2);
nmax  = max(nba,nbw);
XX    = Float64[]  ;  
    
for i=1:nbw
   for j=1:nba
      if i+j-2<=(nmax-1);
         XX=[XX; kron(XW[:,i],XA[:,j])];
      end
   end
end
return XX
end;

In [9]:
f  =function(theta);
RHS=Float64[];
LHS=Float64[];
XX =Float64[];
XX0=Float64[];    
ct = 0.0;    
c1 = 0.0;    
for i   = 1:nodek
    for j=1:nodea
        XX0  = makepoly(XA[j,:]',XK[i,:]');
        ct   = exp.(dot(XX0,theta));
        XX   = [XX;XX0];
        k1   = exp.(at[j])*kt[i].^alpha+(1-delta)*kt[i]-ct;
        rk1  = transfo(log.(k1),kmin,ksup);
        
        vrbk = transpose(collect(0:nbk));    
        xk1  = cheb(complex(rk1),vrbk);      
        a1   = rho*at[j].+(1-rho)*ma.+wgrid;
        ra1  = transfo(a1,amin,asup);           # grid
        
        XA1  = cheb(complex(ra1),vnba);              # Polynomials
        XX1  = makepoly(XA1,xk1);
        XX1  = reshape(XX1,size(XA1,1), size(XA1,2)*size(xk1,2)-3)    
        c1   = exp.(XX1*theta);
        aux  = transpose(wmat)*(beta*(alpha*exp.(a1)*k1.^(alpha-1).+(1-delta)).*c1.^(-sigma))/sqrt(pi);
        RHS  = [RHS;log.(aux)];
        LHS  = [LHS;log.(ct.^(-sigma))];
    end;      
end;
XX  = reshape(XX,size(XX0,1), nodek*nodea)    
resid     = XX*(LHS-RHS);
return vec(resid);
end;


In [10]:
#Pkg.add("NLsolve");
#using NLsolve;
sol = nlsolve(f,a0);
println("Display final rule:",sol.zero)


Display final rule:[0.11912969506532756, 0.49402055617716484, 0.039867077616288854, 0.36198218773066293, -0.10460462503877645, -0.0014782907351818173, 0.00829982024447955, 0.001346857698654657, -0.0008565680170184621, 6.589865927037133e-5, 0.00027186132413384333, -8.723929697959176e-7]


In [11]:
#final
lt  = length(sol.zero);
nb  = 100;
kt  = collect(kmin:(ksup-kmin)/(nb-1):ksup);
rk  = transfo(kt,kmin,ksup);
rk  = vec(rk);
vnbk= transpose(collect(0:nbk));
XT  = cheb(rk,vnbk);
kt  = exp.(kt);
c=[];k1=[];ii=[];
XX0=[];

for i=1:nodea 
XX0 = makepoly(XA[i,:]',XT)
    XX0 = reshape(XX0,size(XT,1), size(XA,2)*size(XT,2)-3) 
    c  = [c; exp.(XX0*sol.zero)];
end; 
c = reshape(c,100, nodea)
for i=1:nodea 
    k1 = [k1; exp.(at[i])*kt.^alpha+(1-delta)*kt.-c[:,i]];
    ii = [ii; exp.(at[i])*kt.^alpha-c[:,i]];
end;
k1 = reshape(k1,100, nodea);
ii = reshape(ii,100, nodea);

In [12]:
k1

100×10 Array{Any,2}:
 0.981757  1.00165  1.04764  1.13276  …  1.80656  2.17082  2.51303  2.72476
 0.997162  1.01717  1.06345  1.14909     1.82702  2.19355  2.53791  2.75099
 1.01282   1.03296  1.07952  1.16569     1.84777  2.21659  2.56311  2.77754
 1.02875   1.04901  1.09586  1.18255     1.86881  2.23992  2.58862  2.8044
 1.04494   1.06533  1.11246  1.19969     1.89014  2.26356  2.61445  2.8316
 1.0614    1.08192  1.12934  1.2171   …  1.91177  2.28751  2.64061  2.85912
 1.07814   1.09878  1.1465   1.23479     1.93371  2.31178  2.66709  2.88699
 1.09515   1.11593  1.16394  1.25276     1.95595  2.33637  2.69391  2.91519
 1.11245   1.13335  1.18166  1.27103     1.97851  2.36129  2.72106  2.94374
 1.13004   1.15107  1.19968  1.28959     2.00139  2.38654  2.74856  2.97264
 1.14792   1.16909  1.21799  1.30845  …  2.02459  2.41212  2.77641  3.0019
 1.1661    1.1874   1.23661  1.32762     2.04811  2.43805  2.80461  3.03152
 1.18459   1.20602  1.25553  1.34709     2.07198  2.46432  2.83317  3.